This notebook is for after DEMs have been run. It performs a number of tasks extracting parameters from result files, organizing, and associating regions with non-DEM parameters (e.g. AR age) determined other ways. The final result is an updated all_targets dictionary with all of this information available for easy reference. This includes:

- Checking none of the DEM solutions rise above any of the loci curves (sanity check).
- Sorting results by interval type (flaring?) which instrumets used, etc. and appending this info to all_targets dictionaries by key.
- Checking which observations of regions have corresponding HARP patches that may be usable + saving HARP parameter values to all_targets dictionaries by key.
- Parsing active region ages from a CSV file and adding that information to the all_targets dictionaries as well. 

DEM result files are located in directories that hold the associated NuSTAR data products (along with many other files, plots, etc).

The structure is:

- key > #Independent keys for each DATE and each POINTING. e.g. if there are two pointings on date ddd, we will have keys ddd_1, ddd_2. 
- - region > #If there are two sources in the FOV, they each have their own directory structure. (IF NOT, THIS LEVEL IS SKIPPED)
- - - time_interval > the result of time interval selection. Sucessful directories include data products, result files, etc. 
                        failed time intervals only have data products.




The original all_targets dictionary is generated in AR_Inventory.ipynb. 

In [1]:
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

import visualize_dem_results as viz
import all_nu_analysis as ana
import HARP_and_age as haa

import pickle
import importlib

In [2]:
targets_file='/Users/jmdunca2/do-dem/reference_files/all_targets_postghost_postshut.pickle'
with open(targets_file, 'rb') as f:
    all_targets = pickle.load(f)

In [3]:
# for key in list(all_targets.keys()):
#     #Save information about the successful time intervals from the TIS run earlier.
#     ana.post_tis_info_dump(key, targets_file)

Let's do a spot check to make sure none of the DEM solutions violate a simple condition: not exceeding the LOCI curves.

In [4]:

keys = all_targets.keys()
bad_eggs_all, bad_eggs_nuaia, bad_eggs_aia = [], [], []
for k in keys:
    print(k)
    bad_eggs_all.extend(ana.check_loci(k, targets_file, searchstring='_MC_'))
    #bad_eggs_nuaia.extend(ana.check_loci(k, targets_file, searchstring='_no_xrt_'))
    #bad_eggs_aia.extend(ana.check_loci(k, targets_file, searchstring='_onlyaia'))

print('Issues found with:')
print(bad_eggs_all)
# print('Issues found with:')
# print(bad_eggs_nuaia)
# print('Issues found with:')
# print(bad_eggs_aia)

01-nov-14_1
01-nov-14_2
19-feb-16
22-apr-16_1


/Users/jmdunca2/do-dem//dodem/all_nu_analysis.py:2802: RuntimeWarning: divide by zero encountered in divide
  all_loci[i,:] =  data['dn_in'][i]/data['trmatrix'][:,i]


22-apr-16_2
26-jul-16_1
27-jul-16_1
26-jul-16_2
10-oct-17
29-may-18_1
29-may-18_2
09-sep-18
10-sep-18
12-apr-19
13-apr-19
29-jan-20
06-jun-20
07-jun-20
08-jun-20
09-jun-20
08-jan-21
14-jan-21
20-jan-21
29-apr-21
03-may-21_1
03-may-21_2
30-jul-21_1
30-jul-21_2
17-nov-21_1
19-nov-21
20-nov-21
22-nov-21_1
22-nov-21_2
03-jun-22_1
03-jun-22_2
06-sep-22
09-dec-22
Issues found with:
[]


The below takes all of the keys, finds result files, and makes a dictionary with lists of result files/times sorted by instrument combination as well as by flaring/non-flaring (for each key/region, and in total). Separately, it identifies cases that fail an event acceptance threshold (accthreshold, in %accepted needed over the interval to qualify) and also makes lists of those (excluding them from the main lists).

It takes about an hour to run for all the results. After, the results dictionary can be loaded for quick reference to lists of types of files. 

It also edits each result file to add extracted DEM parameters! From now on, those are accessible from the result files and do not need to be re-made. 

It takes about an hour.

In [5]:
#makes a sorted dictionary of paths to result files, including sorted lists of time intervals. 
data2 = ana.sorted_resfiles_dict(targets_file, checkacc=False, accthreshold=95)


with open('/Users/jmdunca2/do-dem/reference_files/result_file_dict_postghost_postshut.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data2, f, pickle.HIGHEST_PROTOCOL)

If we've already done the above, we can just load in the result file dictionary.

In [6]:
# with open('/Users/jmdunca2/do-dem/reference_files/result_file_dict_postghost_postshut.pickle', 'rb') as f:
#     data2 = pickle.load(f)

Printing some result statistics - how many results are there for each key, for each instrument combination, for each type of interval?

In [7]:
importlib.reload(ana)
nones, somes, xrts, nxrts, noquiets = [], [], [], [], []
for k in data2.keys():
    print(k)
    if k != 'all regions':
        any, xrt, quiet = ana.print_stats(data2[k])
        print('')

        if not quiet:
            noquiets.append(k)
    
        if not any:
            nones.append(k)
        else:
            somes.append(k)
            if not xrt:
                nxrts.append(k)
            else:
                xrts.append(k)


print('No-result regions: ', nones)
print('')
print('Result regions: ', somes)
print('')

print('XRT regions: ', xrts)
print('')
print('No-XRT regions: ', nxrts)
print('')

print('No quiet time result regions: ', noquiets)

all regions
01-nov-14_1 region_0
Flare-time only-AIA DEMS:  0
Quiet-time only-AIA DEMS:  2
Flare-time AIA+XRT DEMS:  0
Quiet-time AIA+XRT DEMS:  0
Flare-time AIA+NuSTAR DEMS:  0
Quiet-time AIA+NuSTAR DEMS:  2
Flare-time All-instrument DEMS:  0
Quiet-time All-instrument DEMS:  2
Rejected Times:  0

01-nov-14_1 region_1
Flare-time only-AIA DEMS:  0
Quiet-time only-AIA DEMS:  0
Flare-time AIA+XRT DEMS:  0
Quiet-time AIA+XRT DEMS:  0
Flare-time AIA+NuSTAR DEMS:  0
Quiet-time AIA+NuSTAR DEMS:  0
Flare-time All-instrument DEMS:  0
Quiet-time All-instrument DEMS:  0
Rejected Times:  0

01-nov-14_2 region_0
Flare-time only-AIA DEMS:  0
Quiet-time only-AIA DEMS:  0
Flare-time AIA+XRT DEMS:  0
Quiet-time AIA+XRT DEMS:  0
Flare-time AIA+NuSTAR DEMS:  0
Quiet-time AIA+NuSTAR DEMS:  0
Flare-time All-instrument DEMS:  0
Quiet-time All-instrument DEMS:  0
Rejected Times:  0

19-feb-16 region_0
Flare-time only-AIA DEMS:  15
Quiet-time only-AIA DEMS:  10
Flare-time AIA+XRT DEMS:  0
Quiet-time AIA+XRT D

Adding the result file lists to the original targets dictionary:

In [9]:
for kk in all_targets.keys():
    kz = []
    res_file_dicts = []
    for kd in data2.keys():
        if kd[0:len(kk)] == kk[0:len(kk)]:
            kz.append(kd)
            res_file_dicts.append(data2[kd])

    all_targets[kk]['sub_keys'] = kz
    all_targets[kk]['res_file_dict(s)'] = res_file_dicts

with open(targets_file, 'wb') as f:
     # Pickle the 'data' dictionary using the highest protocol available.
     pickle.dump(all_targets, f, pickle.HIGHEST_PROTOCOL)   

Now, we want to add parameters we get from HARP patches (magnetic information) to the target dictionaries as well. We will add all_targets[key]['HARP params'] for each key, a list with entries for each HARP in all_targets[key]['HARP']. For each HARP, there is a dictionary containing chosen HARP parameters (arrays, including an array with the times of each entry). Also in the dictionary is a list of overlapping indices with each NuSTAR orbit. This takes about 4.5 minutes.

In [10]:
all_targets = haa.add_harp_params(all_targets, shush=True)

with open(targets_file, 'wb') as f:
     # Pickle the 'data' dictionary using the highest protocol available.
     pickle.dump(all_targets, f, pickle.HIGHEST_PROTOCOL)   

T_REC
USFLUX
ERRVF
MEANALP
MEANJZH
TOTUSJZ
TOTUSJH
LON_FWT
LAT_FWT
AREA
No result for HARP  0  key:  08-jan-21


Now, we want to add information about the age of each active region at the time that it is being observed. 

In [11]:
agecsv = '/Users/jmdunca2/do-dem/reference_files/Active_Region_Age_Summary.csv'

all_targets = haa.add_AR_ages(all_targets, agecsv)

with open(targets_file, 'wb') as f:
     # Pickle the 'data' dictionary using the highest protocol available.
     pickle.dump(all_targets, f, pickle.HIGHEST_PROTOCOL)  

Let's make an list of the regions that are the same as each other, for later plotting purposes.

In [12]:
#Some of the regions are the same as other regions, and this should be accounted for in the plot. 
samesames = {'same region lists': [
    #['02-sep-15 region_0', '01-sep-15 region_0'],
    #['02-sep-15 region_1'],
    ['01-nov-14_1 region_0'],
    ['01-nov-14_1 region_1'],
    ['19-feb-16 region_0'],
    ['22-apr-16_2 region_0'],
    ['27-jul-16_1 region_0', '26-jul-16_1 region_0'],
    ['26-jul-16_1 region_1'], #'27-jul-16_1 region_1', 
    ['26-jul-16_2 region_0'],
    #['11-sep-17 region_0'],
    #['13-sep-17 region_0', '12-sep-17 region_0'],
    #['13-sep-17 region_1', '12-sep-17 region_1'],
    ['10-oct-17 region_0'],
    ['10-oct-17 region_1'],
    ['29-may-18_1 region_0'],
    ['29-may-18_2 region_0'],
    #['29-may-18_2 region_1'],
    ['10-sep-18 region_0', '09-sep-18 region_0'],
    ['13-apr-19 region_0', '12-apr-19 region_0'],
    ['29-jan-20 region_0'],
    ['08-jun-20 region_0', '07-jun-20 region_0', '06-jun-20 region_0'], #'09-jun-20 region_0',
    ['08-jan-21 region_0'],
    ['14-jan-21 region_0'],
    ['20-jan-21 region_0'],
    ['29-apr-21 region_0'], 
    ['03-may-21_1 region_0'],
    ['03-may-21_2 region_0'],
    #['07-may-21 region_0'],
    #['20-jul-21 region_0'],
    #['20-jul-21 region_1'],
    ['30-jul-21_1 region_0'],
    ['30-jul-21_1 region_1'],
    ['30-jul-21_2 region_0'],
    ['17-nov-21_1 region_0'],
    ['19-nov-21 region_0', '20-nov-21 region_0'], #'22-nov-21_1 region_0'
    ['03-jun-22_2 region_0'],
    ['03-jun-22_2 region_1']#,
    #['06-sep-22 region_0']#,
    #['09-dec-22 region_0']#, 
    #['09-dec-22 region_1'] 
]}



with open('/Users/jmdunca2/do-dem/reference_files/samesames.pickle', 'wb') as f:
     # Pickle the 'data' dictionary using the highest protocol available.
     pickle.dump(samesames, f, pickle.HIGHEST_PROTOCOL)  
    

That should be good for now. 